MCP is defined by Anthropic, the group who made it as an open protocol that standardizes how your LLM applications connect to and work with your tools and data sources.

Simple Analogy : Take example of USB - Type C wire : I can connect my phone to my laptop just by plugging the USB to my laptop and Type C end to my phone. If we didnt have this wire, we would probably download some custom softwares to connect our devices and transport data between the 2.

Creating tools and providing context to different model providers used to look like below

    AI APP 1  -->  Prompt  -->  Tools  -->  Resources
    AI APP 2  -->  Prompt  -->  Tools  -->  Resources
    AI APP 3  -->  Prompt  -->  Tools  -->  Resources
    AI APP 4  -->  Prompt  -->  Tools  -->  Resources

It was a never ending web of API calls and database to connect to your agent for every application you tried to build

Anthropic solves this problem firsthand, which is why they suggested a universal model context protocol for model providers and tool builders to use.

In this standard, the MCP Host(Our Agent) hosts an MCP client, which communicates to the MCP server. In our case, we can think of the MCP host as the AI application or the agent

These MCP servers can expose tools, resources, like read only data or prompt templates to the client.

Once we have built an MCP server with our tools and context, it is very easy to share with other projects or developers streamlining future agent builds.

There is a huge open source community of MCP Servers that other people have built, which we can easily share insert into our agent and other types of AI applications.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import sys
import asyncio

# Fix for Windows issues in Jupyter notebooks
if sys.platform == "win32":
    # 1. Use ProactorEventLoop for subprocess support
    if not isinstance(asyncio.get_event_loop_policy(), asyncio.WindowsProactorEventLoopPolicy):
        asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    
    # 2. Redirect stderr to avoid fileno() error when launching MCP servers
    if "ipykernel" in sys.modules:
        sys.stderr = sys.__stderr__


## Local MCP server

In [4]:
#!pip install langchain-mcp-adapters

In [5]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "local_server": {
                "transport": "stdio",
                "command": "python",
                "args": ["resources/2.1_mcp_server.py"],
            }
    }
)

In [6]:
# get tools
tools = await client.get_tools()

# get resources
resources = await client.get_resources("local_server")

# get prompts
prompt = await client.get_prompt("local_server", "prompt")
prompt = prompt[0].content

In [7]:
tools

[StructuredTool(name='search_web', description='Search the web for information', args_schema={'properties': {'query': {'title': 'Query', 'type': 'string'}}, 'required': ['query'], 'title': 'search_webArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x000001B565FD9580>)]

In [8]:
resources

[Blob 1878611928768]

In [9]:
prompt

'\n    You are a helpful assistant that answers user questions about LangChain, LangGraph and LangSmith.\n\n    You can use the following tools/resources to answer user questions:\n    - search_web: Search the web for information\n    - github_file: Access the langchain-ai repo files\n\n    If the user asks a question that is not related to LangChain, LangGraph or LangSmith, you should say "I\'m sorry, I can only answer questions about LangChain, LangGraph and LangSmith."\n\n    You may try multiple tool and resource calls to answer the user\'s question.\n\n    You may also ask clarifying questions to the user to better understand their question.\n    '

In [10]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-4.1-nano",
    tools=tools,
    system_prompt=prompt
)

In [11]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Tell me about the langchain-mcp-adapters library")]},
    config=config
)

In [12]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Tell me about the langchain-mcp-adapters library', additional_kwargs={}, response_metadata={}, id='fee86c4f-9d1d-4e96-b615-2d8bc64f9a0c'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_TkixeHg9GNmSyzsykNji3DRr', 'function': {'arguments': '{"query":"langchain-mcp-adapters library"}', 'name': 'search_web'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 188, 'total_tokens': 208, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_f0bc439dc3', 'id': 'chatcmpl-Cwk73XEpHiG8Vzo24QOP4bFHvD0mk', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--4a3ff6a5-c173-4826-924b-a48edafb3144-0', tool_calls=[{

## Online MCP

Open MCP Servers
1. https://mcp.so/servers
2. https://github.com/punkpeye/awesome-mcp-servers
3. https://glama.ai/mcp/servers

In [13]:
client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "uvx",
            "args": [
                "mcp-server-time",
                "--local-timezone=America/New_York"
            ]
        }
    }
)

tools = await client.get_tools()

In [14]:
agent = create_agent(
    model="gpt-4.1-nano",
    tools=tools,
)

In [15]:
question = HumanMessage(content="What time is it?")

response = await agent.ainvoke(
    {"messages": [question]}
)

pprint(response)

{'messages': [HumanMessage(content='What time is it?', additional_kwargs={}, response_metadata={}, id='42d32ed4-0553-4084-b55f-ab19f39c2eb8'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_KS0DkUrJkpw0S1p9jBApU0if', 'function': {'arguments': '{"timezone":"America/New_York"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 215, 'total_tokens': 233, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7f8eb7d1f9', 'id': 'chatcmpl-CwkBQb8haLSUkE6JaqqiVmxtJuDiY', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--2d921b31-1f79-41f5-b867-53916c807b07-0', tool_calls=[{'name': 'get_current_time', 'args': {